In [37]:
import torch
import torchvision
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.datasets import CIFAR10
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.models import resnet18

In [40]:
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(512, 10)

    def forward(self, x):
        x = self.resnet(x)
        return x

In [42]:
# 定义类别权重和加权采样器
def calculate_class_weights(targets):
    class_counts = torch.bincount(targets)
    class_weights = 1.0 / class_counts.float()
    return class_weights

In [43]:
# 加载 CIFAR-10并构造为不平衡数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [44]:
# 将目标转换为 Tensor
train_targets = torch.tensor(train_dataset.targets)
test_targets = torch.tensor(test_dataset.targets)

In [45]:
# 计算类别权重
class_weights = calculate_class_weights(train_targets)

# 为每个样本设置权重
train_dataset_weights = [class_weights[class_idx] for class_idx in train_targets]
test_dataset_weights = [class_weights[class_idx] for class_idx in test_targets]

# 定义采样器
train_sampler = WeightedRandomSampler(train_dataset_weights, len(train_dataset))
test_sampler = WeightedRandomSampler(test_dataset_weights, len(test_dataset))

# 创建数据加载器
train_dataloader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler)
test_dataloader = DataLoader(test_dataset, batch_size=64, sampler=test_sampler)

# 初始化模型
model = Net().to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 学习率调度器
scheduler = MultiStepLR(optimizer, milestones=[40, 80], gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/ma-user/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [46]:
# 训练循环
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return running_loss / len(dataloader), 100.0 * correct / total

In [47]:
# 测试循环
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return running_loss / len(dataloader), 100.0 * correct / total

In [48]:
# 训练和测试
num_epochs = 100

for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_dataloader, criterion, optimizer)
    test_loss, test_accuracy = evaluate(model, test_dataloader, criterion)

    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Train Acc: {train_accuracy:.2f}% - Test Loss: {test_loss:.4f} - Test Acc: {test_accuracy:.2f}%')

    scheduler.step()


Epoch 1/100 - Train Loss: 1.0081 - Train Acc: 66.83% - Test Loss: 0.8071 - Test Acc: 72.10%
Epoch 2/100 - Train Loss: 0.6114 - Train Acc: 79.64% - Test Loss: 0.6850 - Test Acc: 77.50%
Epoch 3/100 - Train Loss: 0.4799 - Train Acc: 83.97% - Test Loss: 0.6374 - Test Acc: 79.12%
Epoch 4/100 - Train Loss: 0.3753 - Train Acc: 87.43% - Test Loss: 0.6774 - Test Acc: 79.64%
Epoch 5/100 - Train Loss: 0.2957 - Train Acc: 90.14% - Test Loss: 0.6429 - Test Acc: 80.40%
Epoch 6/100 - Train Loss: 0.2424 - Train Acc: 91.90% - Test Loss: 0.6614 - Test Acc: 80.52%
Epoch 7/100 - Train Loss: 0.2018 - Train Acc: 93.42% - Test Loss: 0.7144 - Test Acc: 80.43%
Epoch 8/100 - Train Loss: 0.1697 - Train Acc: 94.34% - Test Loss: 0.7506 - Test Acc: 79.85%
Epoch 9/100 - Train Loss: 0.1401 - Train Acc: 95.33% - Test Loss: 0.7861 - Test Acc: 80.78%
Epoch 10/100 - Train Loss: 0.1244 - Train Acc: 95.82% - Test Loss: 0.8574 - Test Acc: 79.44%
Epoch 11/100 - Train Loss: 0.1084 - Train Acc: 96.32% - Test Loss: 0.8350 - Tes